# Tarefa 5 - Treinando Modelos Densos Estáticos
## Alunos
- Yago César
- Samuel Morais


## Lendo o corpus pré-processado

In [5]:
import pandas as pd
import ast

df = pd.read_csv("data/TokenizedSentences.csv")

# Transformando as strings "[...strings]" para listas de strings
df['text'] = df['text'].apply(ast.literal_eval)

corpus = df['text'].tolist()
df['text']

0        [nada, queira, voltar, tempo, refazer, vida, t...
1        [lidando, mudanças, humor, tido, muitas, mudan...
2        [noiva, usar, voce, ja, sabe, pergunte, nao, f...
3        [muitas, vezes, consigo, trabalhar, cerca, hor...
4        [quais, ações, sentindo, ansioso, procurando, ...
                               ...                        
50962    [vivo, outras, fazendo, péssimo, trabalho, viv...
50963    [parabéns, amigo, tapa, buraco, depende, quer,...
50964    [ansiedade, doente, epiléptico, ansiedade, def...
50965    [passei, período, difícil, durante, últimos, 3...
50966    [sair, terapia, desde, 14, anos, finalmente, d...
Name: text, Length: 50967, dtype: object

## Treinando o modelo

In [ ]:
from gensim.models import Word2Vec
from time import time
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

model = Word2Vec(
    vector_size=300,
    window=5,
    min_count=5,
    sample=6e-5,
    sg=1 # CBOW = 0 ; SkipGram = 1
)

t = time()
model.build_vocab(corpus, progress_per=10000)

print(f'Time to build vocab: {round((time() - t) / 60, 2)}')

t = time()
model.train(corpus, total_examples=model.corpus_count, epochs=30, report_delay=1)
print(f'Time to train the model: {round((time() - t) / 60, 2)} mins')

model.save('word2vec_sgram.model')


Time to build vocab: 0.01
Time to train the model: 2.21 mins


/tmp/ipykernel_28877/2822979235.py:21: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


## Exploração semântica
### Palavras mais similares

Tomando como base o contexto do corpus, as 4 palavras escohidas foram:
- depressão;
- ansiedade;
- medo;
- suicídio.

In [ ]:
model.wv.most_similar(positive=["depressão"], topn=5)

[('ataques', 0.8192235231399536),
 ('ataque', 0.7734318971633911),
 ('ansiedade', 0.6586445569992065),
 ('pânicoansiedade', 0.5356523990631104),
 ('ansioso', 0.4834514856338501),
 ('sintomas', 0.45120304822921753),
 ('tendo', 0.45071545243263245),
 ('batimentos', 0.4454892873764038),
 ('ansiedadepânico', 0.44178271293640137),
 ('cardíaco', 0.4200853109359741)]

In [14]:
model.wv.most_similar(positive=["ansiedade"], topn=5)

[('depressão', 0.6619756817817688),
 ('pânico', 0.6586444973945618),
 ('social', 0.657738983631134),
 ('ataques', 0.608400821685791),
 ('diagnosticado', 0.5388233661651611)]

In [16]:
model.wv.most_similar(positive=["medo"], topn=5)

[('posso', 0.5136370658874512),
 ('causa', 0.5071543455123901),
 ('morrer', 0.4897680878639221),
 ('preocupado', 0.4877718985080719),
 ('ideia', 0.48593705892562866)]

In [17]:
model.wv.most_similar(positive=["suicídio"], topn=5)

[('matar', 0.6484552621841431),
 ('suicida', 0.5514351725578308),
 ('cometer', 0.5420529842376709),
 ('tentativa', 0.5321749448776245),
 ('morte', 0.5142024755477905)]

## Similaridade semântica entre palavras

Palavras escolhidas:
- ansiedade;
- depressão;
- suicídio.

In [20]:
model.wv.similarity('ansiedade', 'depressão')

0.6619757

In [25]:
model.wv.similarity('suicídio', 'ansiedade')

0.14742094

In [26]:
model.wv.similarity('depressão', 'suicídio')

0.36058834

## Analogia semântica

Analogias escolhidas:
- "suicídio + depressão": entender quais palavras mais se relacionam com a combinação de **"suicídio"** com **"depressão"**.
"depressão + ansiedade - suicídio": .

In [ ]:
# suicídio + depressão
model.wv.most_similar(positive=["suicídio", "depressão"])

[('suicida', 0.6331512331962585),
 ('matar', 0.5774985551834106),
 ('suicidas', 0.5762591361999512),
 ('deprimido', 0.5468285083770752),
 ('mental', 0.5226360559463501),
 ('diagnosticado', 0.5204373598098755),
 ('vida', 0.5046091675758362),
 ('ansiedade', 0.49066269397735596),
 ('transtorno', 0.47831130027770996),
 ('pensamentos', 0.4756019711494446)]

In [29]:
# depressão + ansiedade - suicídio
model.wv.most_similar(positive=["depressão", "ansiedade"], negative=['suicídio'])

[('social', 0.5230732560157776),
 ('pânico', 0.4886881709098816),
 ('ataques', 0.4742621183395386),
 ('diagnosticado', 0.4650210738182068),
 ('transtorno', 0.454802006483078),
 ('estresse', 0.44036513566970825),
 ('recentemente', 0.43963363766670227),
 ('severa', 0.4338925778865814),
 ('sintomas', 0.4259622097015381),
 ('geral', 0.42242783308029175)]

In [30]:
model.wv.most_similar(positive=["psicólogo", "saúde"])

[('mental', 0.6679843664169312),
 ('terapeuta', 0.6156370639801025),
 ('médico', 0.5743861198425293),
 ('psiquiatra', 0.5581804513931274),
 ('terapia', 0.5561575889587402),
 ('consultar', 0.5146474838256836),
 ('ajuda', 0.5139002203941345),
 ('problemas', 0.4710984528064728),
 ('encaminhou', 0.4589638411998749),
 ('ajudar', 0.4547247588634491)]

## Modelo Doc2Vec


In [ ]:
from gensim.models import Doc2Vec